In [1]:
import pandas as pd
import gender_guesser.detector as gender

### Funciones

In [2]:
def is_initial(name):
    return len(name) < 4 and '.' in name

### Genderizacion

In [3]:
d = gender.Detector(case_sensitive=False)

In [4]:
DATA = '../data/'

In [5]:
DATA_SCRAPING = '../scrapping/outputs/authors_data/'

In [10]:
influenced_authors = pd.read_csv(DATA_SCRAPING+'influenced_by_1_from_1.csv')

In [11]:
influenced_authors.head()

,author_id,id,name,score
0,1726246,145645850,Ling Ren,411
1,1726246,2012099,Christopher W. Fletcher,357
2,1726246,1736809,Kartik Nayak,299
3,1726246,2155148,Attila A. Yavuz,266
4,1726246,144568986,T.-H. Hubert Chan,247


In [12]:
len(influenced_authors)

129017

In [13]:
influenced_authors['first_name'] = influenced_authors.name.apply(lambda x: x.split()[0])

In [14]:
influenced_authors.head()

,author_id,id,name,score,first_name
0,1726246,145645850,Ling Ren,411,Ling
1,1726246,2012099,Christopher W. Fletcher,357,Christopher
2,1726246,1736809,Kartik Nayak,299,Kartik
3,1726246,2155148,Attila A. Yavuz,266,Attila
4,1726246,144568986,T.-H. Hubert Chan,247,T.-H.


In [15]:
influenced_authors.sample(10)

,author_id,id,name,score,first_name
19215,2387132,2347370,Johannes Hötzer,169,Johannes
100032,145129647,144566689,Belén Calvo,103,Belén
103267,1946269,3127133,Hassan Ghassemian,18,Hassan
75355,143947197,153866711,Sebastián Ventura,26,Sebastián
6864,3308554,9241059,Saeed Ebadi,16,Saeed
118061,49424792,1403699864,Nathan Souviraà-Labastie,8,Nathan
39746,144426849,1693689,Nachiappan Nagappan,103,Nachiappan
40834,50252101,102151391,Silvia Ayunda Murad,5,Silvia
95061,40771260,15454616,Alexander A. Konovalenko,16,Alexander
107892,79573587,3379148,Federico Pallottino,6,Federico


## Creo 'first_name'

In [16]:
influenced_authors[influenced_authors['first_name'].apply(lambda x: len(x)<4)]

,author_id,id,name,score,first_name
37,35964168,144688379,Dan Feng,108,Dan
60,143859272,143966172,Pan Hui,113,Pan
93,2153304,1709353,Amr El Abbadi,35,Amr
108,1702744,1752788,Dan Boneh,96,Dan
162,2478475,1697247,Ali Sunyaev,16,Ali
...,...,...,...,...,...
128939,3196491,144964282,D. Richard Kuhn,25,D.
128953,2747138,144687982,K. Gopakumar,80,K.
128954,2747138,144286539,K. Gopakumar,51,K.
128976,1409839496,1397306793,Amr El-Mougy,1,Amr


## Detecto y descarto iniciales en 'first_name'

In [17]:
influenced_authors['is_initial'] = influenced_authors.first_name.apply(is_initial)
influenced_authors.is_initial.value_counts()

False    126586
True       2431
Name: is_initial, dtype: int64

In [18]:
influenced_authors.head()

,author_id,id,name,score,first_name,is_initial
0,1726246,145645850,Ling Ren,411,Ling,False
1,1726246,2012099,Christopher W. Fletcher,357,Christopher,False
2,1726246,1736809,Kartik Nayak,299,Kartik,False
3,1726246,2155148,Attila A. Yavuz,266,Attila,False
4,1726246,144568986,T.-H. Hubert Chan,247,T.-H.,False


In [19]:
influenced_authors = influenced_authors[~ influenced_authors.is_initial]

In [20]:
len(influenced_authors)

126586

## Genderizo archivo para id unicos

In [21]:
influenced_authors_uniques = influenced_authors[['id', 'first_name']].drop_duplicates()

In [22]:
len(influenced_authors_uniques)

80153

In [23]:
influenced_authors_uniques.head()

,id,first_name
0,145645850,Ling
1,2012099,Christopher
2,1736809,Kartik
3,2155148,Attila
4,144568986,T.-H.


In [24]:
influenced_authors_uniques['gender'] = influenced_authors_uniques.first_name.apply(lambda x: d.get_gender(x))

In [25]:
influenced_authors_uniques.head()

,id,first_name,gender
0,145645850,Ling,andy
1,2012099,Christopher,male
2,1736809,Kartik,unknown
3,2155148,Attila,male
4,144568986,T.-H.,unknown


## Mergeo con archivo original

In [26]:
influenced_authors = influenced_authors.merge(influenced_authors_uniques, on=['id', 'first_name'], how='left')

In [27]:
influenced_authors[influenced_authors.gender == 'unknown']

,author_id,id,name,score,first_name,is_initial,gender
2,1726246,1736809,Kartik Nayak,299,Kartik,False,unknown
4,1726246,144568986,T.-H. Hubert Chan,247,T.-H.,False,unknown
11,144066998,2913151,Mayutan Arumaithurai,134,Mayutan,False,unknown
22,1726246,1736809,Kartik Nayak,299,Kartik,False,unknown
24,1726246,144568986,T.-H. Hubert Chan,247,T.-H.,False,unknown
...,...,...,...,...,...,...,...
126549,3415759,1388632905,Nhien-An Le-Khac,5,Nhien-An,False,unknown
126550,3415759,51118223,Ulia Trifonova,3,Ulia,False,unknown
126552,1786771,1786905,Gabor Aranyi,36,Gabor,False,unknown
126554,1786771,1759275,Gilan Jackont,26,Gilan,False,unknown


##  Corroboro que no haya nulls en 'gender'

In [28]:
influenced_authors[influenced_authors.gender.isnull()]

,author_id,id,name,score,first_name,is_initial,gender


## Muestreo de distribucion de genero

In [29]:
influenced_authors_groupped = influenced_authors.groupby('gender').count()
influenced_authors_groupped

,author_id,id,name,score,first_name,is_initial
gender,,,,,,
andy,8961,8961,8961,8961,8961,8961
female,19713,19713,19713,19713,19713,19713
male,75319,75319,75319,75319,75319,75319
mostly_female,1001,1001,1001,1001,1001,1001
mostly_male,2830,2830,2830,2830,2830,2830
unknown,18762,18762,18762,18762,18762,18762


In [30]:
influenced_authors_groupped['gender_rate'] = round((influenced_authors_groupped['author_id']*100)/(influenced_authors_groupped['author_id'].sum()),2)

In [31]:
influenced_authors_groupped = influenced_authors_groupped[['author_id', 'gender_rate']].rename(columns={'author_id':'count'})

In [32]:
influenced_authors_groupped

,count,gender_rate
gender,,
andy,8961,7.08
female,19713,15.57
male,75319,59.50
mostly_female,1001,0.79
mostly_male,2830,2.24
unknown,18762,14.82


## Exporto a csv

In [ ]:
# influenced_authors[['author_id', 'id', 'name', 'score', 'gender']].to_csv(DATA+'influencer_author_id_name_gender.csv', index=False)

In [33]:
influenced_authors[['author_id', 'id', 'name', 'score', 'gender']].to_csv(DATA+'influenced_by_author_id_name_gender.csv', index=False)